In [1]:
%pip install mlflow scikit-learn==1.5.0

Note: you may need to restart the kernel to use updated packages.


In [2]:
import digitalhub as dh
import pandas as pd
import os

In [3]:
PROJECT = "demo-ml"
project = dh.get_or_create_project(PROJECT)

In [4]:
%%writefile train-model.py

from digitalhub_runtime_python import handler

from digitalhub_ml.entities.utils import from_mlflow_run
import mlflow

from sklearn import datasets, svm
from sklearn.model_selection import GridSearchCV

@handler()
def train(project):
    mlflow.sklearn.autolog(log_datasets=True)

    iris = datasets.load_iris()
    parameters = {"kernel": ("linear", "rbf"), "C": [1, 10]}
    svc = svm.SVC()
    clf = GridSearchCV(svc, parameters)

    clf.fit(iris.data, iris.target)
    run_id = mlflow.last_active_run().info.run_id

    # utility to map mlflow run artifacts to model metadata
    model_params = from_mlflow_run(run_id)
    
    project.log_model(
        name="model-mlflow",
        kind="mlflow",
        **model_params
)

Overwriting train-model.py


In [5]:
train_fn = project.new_function(
                         name="train",
                         kind="python",
                         python_version="PYTHON3_9",
                         source={"source": "train-model.py", "handler": "train"},
                         requirements=["scikit-learn==1.5.0", "mlflow==2.15.1"])

In [6]:
train_run = train_fn.run(action="job", local_execution=False)

In [7]:
model = project.get_model("model-mlflow")
model.spec.path

's3://datalake/demo-ml/model/model-mlflow/1283563a-23a6-4c12-9113-e77014519944/'

In [8]:
func = project.new_function(name="serve_mlflowmodel",
                            kind="mlflowserve",
                            model_name="testmodel",
                            path=model.spec.path + 'model')

serve_run = func.run(action="serve")

In [10]:
SERVICE_URL = serve_run.refresh().status.to_dict()["service"]["url"]
MODEL_NAME = "testmodel"

In [11]:
from sklearn import datasets

iris = datasets.load_iris()
test_input = iris.data[0:2].tolist()
test_input

[[5.1, 3.5, 1.4, 0.2], [4.9, 3.0, 1.4, 0.2]]

In [12]:
import requests

with requests.post(f'http://{SERVICE_URL}/v2/models/{MODEL_NAME}/infer', json={
    "inputs": [
        {
        "name": "input-0",
        "shape": [2, 4],
        "datatype": "FP64",
        "data": test_input
        }
    ]
}) as r:
    res = r.json()
print(res)

{'model_name': 'testmodel', 'id': '8632b95d-5eb3-47c3-a6f3-a6a7a416be95', 'parameters': {'content_type': 'np'}, 'outputs': [{'name': 'output-1', 'shape': [2, 1], 'datatype': 'INT64', 'parameters': {'content_type': 'np'}, 'data': [0, 0]}]}
